## 철새도래지와 AI 

In [1]:
!pip install folium

In [2]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MiniMap

In [3]:
# 지도 생성하기
migrate habitats = [37.5536067,126.9674308] 
m = folium.Map(location=seoul_station,
               zoom_start=12)

# 마커 클러스터 추가하기
marker_cluster = MarkerCluster().add_to(m)  

# 미니맵 추가하기
minimap = MiniMap() 
minimap.add_to(m)

# 위치 하나씩 추가하기
for i in raw.index:
    name = raw.loc[i, 'place_name']
    long = raw.loc[i, 'x']  # 경도
    lat = raw.loc[i, 'y']   # 위도
    folium.Marker([lat, long],tooltip= name).add_to(marker_cluster)  

SyntaxError: invalid syntax (<ipython-input-3-aa063cb5fc3c>, line 2)

## 가정: 철새도래지에서 멀어질수록 발생(시.군.구) 보다 비발생(시.군.구)가 더 많다
### 귀무가설: 철새도래지에 따른 비발생(시.군.구), 발생(시.군.구) 차이가 없다.
### 대립가설: 철새도래지에 따른 비발생(시.군.구), 발생(시.군.구) 차이가 있다.

가설검정 과정
### 1. 데이터 정제하기
-비발생 데이터 시.군.구 경영체 수 합계 만들기(각 년도 별) [비발생농가: 등록현황-발생농가]

-발생농가 데이터 시.군.구 발생건수 합계 만들기(각 년도 별)

-철새도래지 주소 엑셀로 정리하기

-네이버 맵 api를 이용하여 발생,비발생 시.군.구 철새도래지 거리 데이터프레임 만들기(컬럼: 하천)

### 2. 데이터 분석
-하천 별로 뽑아서 가까운 순으로 정렬

-정렬된 데이터 컬럼 [발생/비발생(등록현황-발생농가)] 수렴 or 발산 확인하기

-선형회귀 기울기 값 구하기(하천에 대한 기울기값 구하기)

-신뢰구간 구하기.

In [1]:
import pandas as pd
import numpy as np
import datetime
import pandas as pd
import seaborn as sns
import scipy as sp
from scipy import stats

In [170]:
# 년도 별 가금농가 등록 현황 파일 불러오기

AI_co_15=pd.read_csv("./data/local_breeding_2015.csv")
AI_co_15=AI_co_15.drop(0,axis=0)
AI_co_16=pd.read_csv("./data/local_breeding_2016.csv")
AI_co_16=AI_co_16.drop(0,axis=0)
AI_co_17=pd.read_csv("./data/local_breeding_2017.csv")
AI_co_17=AI_co_17.drop(0,axis=0)
AI_co_18=pd.read_csv("./data/local_breeding_2018.csv")
AI_co_18=AI_co_18.drop(0,axis=0)
AI_co_19=pd.read_csv("./data/local_breeding_2019.csv")
AI_co_19=AI_co_19.drop(0,axis=0)
AI_co_20=pd.read_csv("./data/local_breeding_2020.csv")
AI_co_20=AI_co_20.drop(0,axis=0)

In [171]:
#타입 변환

#AI_co_15['시도']=AI_co_15['시도'].astype('str')

In [172]:
AI_co_15.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 588 entries, 1 to 588
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   년도           588 non-null    float64
 1   주소           588 non-null    object 
 2   시도           588 non-null    object 
 3   시군           587 non-null    object 
 4   축종명          586 non-null    object 
 5   경영체수 합계(건)   588 non-null    float64
 6   시설여부(경영체 수)  588 non-null    object 
 7   Unnamed: 7   588 non-null    object 
 8   경영형태(시설 개수)  588 non-null    object 
 9   Unnamed: 9   588 non-null    object 
 10  사육 규모        588 non-null    float64
 11  단위           588 non-null    object 
dtypes: float64(3), object(9)
memory usage: 59.7+ KB


In [173]:
AI_co_15['년도']=AI_co_15['년도'].astype('str')
AI_co_15.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 588 entries, 1 to 588
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   년도           588 non-null    object 
 1   주소           588 non-null    object 
 2   시도           588 non-null    object 
 3   시군           587 non-null    object 
 4   축종명          586 non-null    object 
 5   경영체수 합계(건)   588 non-null    float64
 6   시설여부(경영체 수)  588 non-null    object 
 7   Unnamed: 7   588 non-null    object 
 8   경영형태(시설 개수)  588 non-null    object 
 9   Unnamed: 9   588 non-null    object 
 10  사육 규모        588 non-null    float64
 11  단위           588 non-null    object 
dtypes: float64(2), object(10)
memory usage: 59.7+ KB


In [174]:
AI_co_15['년도']=AI_co_15['년도'].map(lambda x:x.replace('.0',''))

In [176]:
AI_co_15['년도']=AI_co_15['년도'].astype('int')

#날짜 float 타입으로 오류
AI_co_15['datetime'] =AI_co_15['년도'].apply(lambda x: pd.to_datetime(str(x), format='%Y'))
AI_co_15

In [178]:
AI_r15=AI_co_15.groupby(['년도','주소']).sum()
AI_r16=AI_co_16.groupby(['년도','주소']).sum()
AI_r17=AI_co_17.groupby(['년도','주소']).sum()
AI_r18=AI_co_18.groupby(['년도','주소']).sum()
AI_r19=AI_co_19.groupby(['년도','주소']).sum()
AI_r20=AI_co_20.groupby(['년도','주소']).sum()

In [179]:
AI_r15

경영체수 합계(건)      사육 규모
년도   주소                            
2015 강원도강릉시        134.0   158892.0
     강원도고성군         34.0   108268.0
     강원도동해시         25.0    73928.0
     강원도삼척시         49.0    42086.0
     강원도속초시         10.0     1522.0
...                  ...        ...
     충청북도제천시       107.0   847705.0
     충청북도증평군        11.0   236250.0
     충청북도진천군        88.0  2400796.0
     충청북도청주시       147.0  2725985.0
     충청북도충주시        88.0  3970062.0

[197 rows x 2 columns]

In [157]:
AI_r15['년도'].

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년도          197 non-null    float64
 1   주소          197 non-null    object 
 2   경영체수 합계(건)  197 non-null    float64
 3   사육 규모       197 non-null    float64
dtypes: float64(3), object(1)
memory usage: 6.3+ KB


In [63]:
# 각 년도 별 등록현황 합치기
AI_r15.reset_index(inplace=True)
AI_r16.reset_index(inplace=True)
AI_r17.reset_index(inplace=True)
AI_r18.reset_index(inplace=True)
AI_r19.reset_index(inplace=True)
AI_r20.reset_index(inplace=True)

AI_migrate =pd.concat([AI_r15,AI_r16,AI_r17,AI_r18,AI_r19,AI_r20], sort=True)

In [66]:
AI_migrate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1145 entries, 0 to 189
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   경영체수 합계(건)  1145 non-null   float64
 1   년도          1145 non-null   float64
 2   사육 규모       1145 non-null   float64
 3   주소          1145 non-null   object 
dtypes: float64(3), object(1)
memory usage: 44.7+ KB


In [88]:
AI_migrate=AI_migrate.reindex(columns=['년도','주소','경영체수 합계(건)','사육 규모'])
AI_migrate.head()

,년도,주소,경영체수 합계(건),사육 규모
0,2015.0,강원도강릉시,134.0,158892.0
1,2015.0,강원도고성군,34.0,108268.0
2,2015.0,강원도동해시,25.0,73928.0
3,2015.0,강원도삼척시,49.0,42086.0
4,2015.0,강원도속초시,10.0,1522.0


In [136]:
#발생농가 확인

AI=pd.read_csv("HPAIOccurlist_final.csv")
AI_occur=AI.copy()

In [137]:
AI_occur.head()

,주소,축종,사육수,진단일,발생일,지역코드
0,충청북도 음성군 삼성면 용성리,닭-육계,24000,20031212,20031212,127
1,충청북도 음성군 삼성면 덕정리,오리,3200,20031217,20031217,127
2,충청북도 음성군 삼성면 용성리,닭,15000,20031219,20031219,127
3,경상북도 경주시 안강읍 육통리,닭-산란계,10650,20031221,20031221,138
4,전라남도 나주시 산포면 매성리,오리,14900,20031221,20031221,156


In [138]:
AI_occur['datetime'] =AI_occur['발생일'].apply(lambda x: pd.to_datetime(str(x), format='%Y/'))
AI_occur.head()

,주소,축종,사육수,진단일,발생일,지역코드,datetime
0,충청북도 음성군 삼성면 용성리,닭-육계,24000,20031212,20031212,127,2003-12-12
1,충청북도 음성군 삼성면 덕정리,오리,3200,20031217,20031217,127,2003-12-17
2,충청북도 음성군 삼성면 용성리,닭,15000,20031219,20031219,127,2003-12-19
3,경상북도 경주시 안강읍 육통리,닭-산란계,10650,20031221,20031221,138,2003-12-21
4,전라남도 나주시 산포면 매성리,오리,14900,20031221,20031221,156,2003-12-21


In [139]:
# DataFrame의 index 변형하기
AI_occur.set_index('datetime', inplace=True) 
AI_occur

,주소,축종,사육수,진단일,발생일,지역코드
datetime,,,,,,
2003-12-12,충청북도 음성군 삼성면 용성리,닭-육계,24000,20031212,20031212,127
2003-12-17,충청북도 음성군 삼성면 덕정리,오리,3200,20031217,20031217,127
2003-12-19,충청북도 음성군 삼성면 용성리,닭,15000,20031219,20031219,127
2003-12-21,경상북도 경주시 안강읍 육통리,닭-산란계,10650,20031221,20031221,138
2003-12-21,전라남도 나주시 산포면 매성리,오리,14900,20031221,20031221,156
...,...,...,...,...,...,...
2021-03-21,전라남도 장흥군 장평면 기동리,오리-육용오리,20000,20210322,20210321,260
2021-03-22,전라남도 장흥군 장평면 기동리,오리-육용오리,11800,20210323,20210322,260
2021-03-23,전라남도 나주시 산포면 등수리,오리-육용오리,19000,20210324,20210323,156


In [140]:
AI_occur['년도']=AI_occur.index.year
AI_occur

,주소,축종,사육수,진단일,발생일,지역코드,년도
datetime,,,,,,,
2003-12-12,충청북도 음성군 삼성면 용성리,닭-육계,24000,20031212,20031212,127,2003
2003-12-17,충청북도 음성군 삼성면 덕정리,오리,3200,20031217,20031217,127,2003
2003-12-19,충청북도 음성군 삼성면 용성리,닭,15000,20031219,20031219,127,2003
2003-12-21,경상북도 경주시 안강읍 육통리,닭-산란계,10650,20031221,20031221,138,2003
2003-12-21,전라남도 나주시 산포면 매성리,오리,14900,20031221,20031221,156,2003
...,...,...,...,...,...,...,...
2021-03-21,전라남도 장흥군 장평면 기동리,오리-육용오리,20000,20210322,20210321,260,2021
2021-03-22,전라남도 장흥군 장평면 기동리,오리-육용오리,11800,20210323,20210322,260,2021
2021-03-23,전라남도 나주시 산포면 등수리,오리-육용오리,19000,20210324,20210323,156,2021


In [144]:
AI_occur_year=AI_occur.groupby(['년도','주소'])['발생일'].count()
AI_occur_year

년도    주소                 
2003  경기도 이천시 율면 신추리         2
      경상북도 경주시 안강읍 육통리       4
      울산광역시 울주군 상북면 명촌리      2
      전라남도 나주시 산포면 매성리       2
      충청남도 천안시 동남구 북면 운용리    1
                            ..
2021  충청북도 음성군 삼성면 용대리       1
      충청북도 음성군 생극면 임곡리       1
      충청북도 음성군 원남면 보룡리       1
      충청북도 충주시 동량면 대전리       1
      충청북도 충주시 소태면 주치리       2
Name: 발생일, Length: 786, dtype: int64

In [146]:
AI_occur_year.to_frame()

발생일
년도   주소                      
2003 경기도 이천시 율면 신추리         2
     경상북도 경주시 안강읍 육통리       4
     울산광역시 울주군 상북면 명촌리      2
     전라남도 나주시 산포면 매성리       2
     충청남도 천안시 동남구 북면 운용리    1
...                       ...
2021 충청북도 음성군 삼성면 용대리       1
     충청북도 음성군 생극면 임곡리       1
     충청북도 음성군 원남면 보룡리       1
     충청북도 충주시 동량면 대전리       1
     충청북도 충주시 소태면 주치리       2

[786 rows x 1 columns]

In [150]:
#발생일자 합 
AI_occur_year=AI_occur_year.reset_index()
AI_occur_year

,index,년도,주소,발생일
0,0,2003,경기도 이천시 율면 신추리,2
1,1,2003,경상북도 경주시 안강읍 육통리,4
2,2,2003,울산광역시 울주군 상북면 명촌리,2
3,3,2003,전라남도 나주시 산포면 매성리,2
4,4,2003,충청남도 천안시 동남구 북면 운용리,1
...,...,...,...,...
781,781,2021,충청북도 음성군 삼성면 용대리,1
782,782,2021,충청북도 음성군 생극면 임곡리,1
783,783,2021,충청북도 음성군 원남면 보룡리,1
784,784,2021,충청북도 충주시 동량면 대전리,1


In [152]:
AI_home=pd.read_csv("./data/migrate habitats.csv")
AI_home

,주소,하천
0,서울특별시 강서구,강서지구
1,서울특별시 중랑구,중랑천
2,경기도 군포시,안양천
3,경기도 용인시,탄천
4,경기도 수원시,원천리천
...,...,...
98,경상남도 양산시,낙동강(원동)
99,제주특별자치도 제주시,하도리
100,제주특별자치도 제주시,용수저수지
101,제주특별자치도 서귀포시,오조리(성산포)


In [155]:
네이버 api 사용해보삼.

SyntaxError: invalid syntax (<ipython-input-155-c2b08398effb>, line 1)